In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import minimize , basinhopping
from scipy.stats import poisson

In [2]:
header = ['trade_id','price','quantity','order_id','timestamp','is_buyer_maker']
df = pd.read_csv('./BTCUSDT-trades-2023-05-31.csv',header=0 , names = header)
df.head()

,trade_id,price,quantity,order_id,timestamp,is_buyer_maker
0,3765419375,27680.0,0.400,11072.0000,1685491200110,True
1,3765419376,27680.0,0.006,166.0800,1685491200156,True
2,3765419377,27680.0,0.300,8304.0000,1685491200157,True
3,3765419378,27680.1,0.003,83.0403,1685491200244,False
4,3765419379,27680.1,0.003,83.0403,1685491203809,False


In [5]:
def calculate_vpin(df, window_size):
    signed_quantity = df['quantity'].values * np.where(df['is_buyer_maker'].values, 1, -1)
    cumulative_volume = np.cumsum(signed_quantity)
    signed_volume = np.cumsum(signed_quantity)

    vpin_values = []

    for i in range(window_size, len(df)):
        volume_in_window = signed_volume[i - window_size:i]
        total_volume = np.sum(np.abs(volume_in_window))
        buyer_volume = np.sum(volume_in_window[volume_in_window > 0])
        seller_volume = np.sum(volume_in_window[volume_in_window < 0])

        if total_volume > 0:
            vpin = (buyer_volume - seller_volume) / total_volume
        else:
            vpin = 0

        vpin_values.append(vpin)

    df['vpin'] = np.concatenate((np.full(window_size, np.nan), vpin_values))
    return df

In [6]:
# VPIN을 계산합니다. 윈도우 크기는 100으로 설정합니다.
window_size = 100
df = calculate_vpin(df, window_size)